# How to build a simple LLM App with LangChain
* Very simple LLM App: some prompting and a single LLM call.

## Goal of the App
* Translate text from English into another language.

## Concepts included
* Monitor with LangSmith.
* Connect with a LLM.
* Use a Prompt Template.
* Use an Output Parser.
* Chain the Prompt Template, the LLM call and the Output Parser.
* Deploy with LangServe.

## Setup

#### Recommended: create new virtualenv
* mkdir your_project_name
* cd your_project_name
* pyenv virtualenv 3.11.4 your_venv_name
* pyenv activate your_venv_name
* pip install jupyterlab
* jupyter lab

In [25]:
#!pip install python-dotenv

#### .env File
Remember to include:
OPENAI_API_KEY=your_openai_api_key

LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
LANGCHAIN_API_KEY=your_langchain_api_key
LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **simpleTranslator**.

In [26]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

In [27]:
#!pip install langchain

## Connect with an LLM and ask it to translate  a sentence from English to Spanish

In [28]:
#!pip install -qU langchain-openai

The `-qU` flag in the `pip install -qU langchain-openai` command is not necessary, although since the LangChain team seems to like it we will explain it here.

It combines two separate options for the `pip` installer:

1. `-q` (or `--quiet`): This option reduces the amount of output that `pip` generates during the installation process. Using `-q` reduces output to show only errors, and using it twice (`-qq`) will suppress output entirely, except for errors.

2. `-U` (or `--upgrade`): This option tells `pip` to upgrade the specified package to the latest available version. It also upgrades its dependencies to the latest versions that are compatible with other installed packages.

So, `pip install -qU langchain-openai` means "install or upgrade the package `langchain-openai` quietly, showing minimal output unless there are errors."

#### OpenAI LLM options
* See [OpenAI website](https://platform.openai.com/docs/models/gpt-3-5-turbo).
* For this project, we will use gpt-3.5-turbo

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

* System Message: defines the role played by the LLM.
* Human Message: the user input.

In [30]:
from langchain_core.messages import HumanMessage, SystemMessage

messagesToTheLLM = [
    SystemMessage(content="Translate the following from English into Spanish"),
    HumanMessage(content="Generative AI is the greatest value-creation opportunity in Human History."),
]

#### Step 1: call the LLM

* `invoke` is the most basic way to call the LLM. We will see more ways of doing it soon.

In [31]:
llm.invoke(messagesToTheLLM)

AIMessage(content='La inteligencia artificial generativa es la mayor oportunidad de creación de valor en la historia humana.', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 32, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-39ae644f-4c3a-4c87-897d-34631695fb86-0')

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Use an Output Parser to format the response of the LLM as a string of text

In [32]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

#### Step 1: call the LLM

In [33]:
initialResponse = llm.invoke(messagesToTheLLM)

#### Step 2: apply the parser to format the output

In [34]:
formattedResponse = parser.invoke(initialResponse)

In [35]:
print(formattedResponse)

La inteligencia artificial generativa es la mayor oportunidad de creación de valor en la historia humana.


## Chain the LLM Call and the Output Parser

In [36]:
llmAndParserChained = llm | parser

#### Steps 1 and 2: call the LLM and apply the parser to the output

In [37]:
responseFromChain=llmAndParserChained.invoke(messagesToTheLLM)

In [38]:
print(responseFromChain)

La inteligencia artificial generativa es la mayor oportunidad de creación de valor en la historia humana.


## Use a Prompt Template with a System Message and Variables
* variables:
    * text_input: the text the user wants to translate 
    * language: the language the user wants to translate the text_input into.

In [39]:
from langchain_core.prompts import ChatPromptTemplate

In [43]:
system_template = "Translate the following text input into {language}:"

In [46]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template), 
        ("user", "{text_input}")
    ]
)

In [58]:
SecondChain = prompt_template | llm | parser

In [59]:
ResponseFromTheLLM = SecondChain.invoke(
    {
    "language": "Spanish",
    "text_input": "Now is the moment to learn Generative AI!"
    }
)

In [60]:
print(ResponseFromTheLLM)

¡Ahora es el momento de aprender la IA Generativa!


## Deploy the App using LangServe

In [61]:
#!pip install "langserve[all]"

* See the file simpleTranslator.py in your code editor.
* Remember to create .gitignore file and include .env there.

In [64]:
# from fastapi import FastAPI
# from langserve import add_routes
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser
# from langchain_openai import ChatOpenAI
# from dotenv import load_dotenv, find_dotenv

# import os

# _ = load_dotenv(find_dotenv())

# openai_api_key = os.environ["OPENAI_API_KEY"]

# llm = ChatOpenAI(model="gpt-3.5-turbo")

# parser = StrOutputParser()

# system_template = "Translate the following into {language}:"

# prompt_template = ChatPromptTemplate.from_messages([
#     ('system', system_template),
#     ('user', '{text}')
# ])

# chain = prompt_template | llm | parser

# app = FastAPI(
#   title="simpleTranslator",
#   version="1.0",
#   description="A simple API server using LangChain's Runnable interfaces",
# )

# add_routes(
#     app,
#     chain,
#     path="/chain",
# )

# if __name__ == "__main__":
#     import uvicorn

#     uvicorn.run(app, host="localhost", port=8000)

#### Run the server
* In terminal:
    * python simpleTranslator.py
* Check the app in the browser:
    * [http://localhost:8000/](http://localhost:8000/)
    * Nothing fancy there... yet.

#### Now let's try the app using the "toy demo UI" provided by LangServe Playground
* Try the app in the browser:
    * [http://localhost:8000/chain/playground/](http://localhost:8000/chain/playground/) 

#### And finally, let's see how we can use the LangServe Playground from the notebook

In [63]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "Spanish", "text": "Generative AI is a bigger opportunity than Internet"})

'La inteligencia artificial generativa es una oportunidad más grande que Internet.'